## Project

#### Due Dates:

11/22 - Project Proposal: Title and one paragraph description. If a team effort, who is on the team.  
12/14, 11:59 pm - Project Due

#### Description

Team or individual

It counts as 15% if your grade.

Combination of writeup and code

Must be in a jupyter notebook with markdown sections for the text and code sections for code and output

You choose the topic but you must either use XGBoost or Deep Learning in the project. You can use other methods in addition to XGBoost or Deep Learning, e.g. comparing different methods to determine the best one. If you do want to use something else see me before 11/22 for approval.

Pretend you are writing a paper for a conference but including all the code in the paper.

#### Suggested Sections

    Introduction
        Goal of your model
        Relevance of topic or method
    Background
        Other similar models on this topic
        Information on method your using
    Data 
        Description
        Visualization
        Data Preparation
    Model
        Hyperparameters & Tuning
        Training
        Testing
    Results
        Visualization
        Degree of success
    Problems & Future Work
    
#### Grading
    70% Code 
    30% Write up
        
    
#### Some suggestions:

Kaggle Competitions, active or completed
    https://www.kaggle.com/competitions  
    
Natural Language Processing
    Sequence Processing
    
Vision
    Face Recognition
    Object Segmentation
    
Recommendation Systems

Search for Machine Learning Projects, Deep Learning Projects
    
   


In [1]:
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.losses import binary_crossentropy
from tensorflow.keras.optimizers import Adam
from gensim.models import KeyedVectors, Word2Vec
from gensim.test.utils import common_texts, get_tmpfile
import pandas as pd
import numpy as np
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords 
from nltk.stem import WordNetLemmatizer
import string
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE

nltk.download('wordnet')

C:\Users\steeni\Anaconda3\envs\tf2\lib\site-packages\tensorflow\python\framework\dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\steeni\Anaconda3\envs\tf2\lib\site-packages\tensorflow\python\framework\dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\steeni\Anaconda3\envs\tf2\lib\site-packages\tensorflow\python\framework\dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\steeni\Anaconda3\envs\tf2\lib\site-packages\tensorflow\python\frame

True

In [2]:
dataset = pd.read_csv('ISEAR.csv').iloc[:, :-1]
dataset.isnull().sum()

Sentiment    0
Sentence     0
dtype: int64

In [3]:
dataset.head()

,Sentiment,Sentence
0,joy,On days when I feel close to my partner and ot...
1,fear,Every time I imagine that someone I love or I ...
2,anger,When I had been obviously unjustly treated and...
3,sadness,When I think about the short time that we live...
4,disgust,At a gathering I found myself involuntarily si...


In [4]:
emotion_labels = {'joy': 0, 'fear': 1, 'anger': 2, 'sadness': 3, 'disgust': 4, 'shame': 5, 'guilt': 6}

In [5]:
emb_model = KeyedVectors.load_word2vec_format("GoogleNews-vectors-negative300.bin", binary=True)
# emb_model = KeyedVectors.load_word2vec_format("lexvec.enwiki+newscrawl.300d.W+C.pos.vectors", binary=False)

In [6]:
def display_closestwords_tsnescatterplot(model, word):
    
    arr = np.empty((0,300), dtype='f')
    word_labels = [word]

    # get close words
    close_words = model.similar_by_word(word)
    
    # add the vector for each of the closest words to the array
    arr = np.append(arr, np.array([model[word]]), axis=0)
    for wrd_score in close_words:
        wrd_vector = model[wrd_score[0]]
        word_labels.append(wrd_score[0])
        arr = np.append(arr, np.array([wrd_vector]), axis=0)
        
    # find tsne coords for 2 dimensions
    tsne = TSNE(n_components=2, random_state=0)
    np.set_printoptions(suppress=True)
    Y = tsne.fit_transform(arr)

    x_coords = Y[:, 0]
    y_coords = Y[:, 1]
    # display scatter plot
    plt.scatter(x_coords, y_coords)

    for label, x, y in zip(word_labels, x_coords, y_coords):
        plt.annotate(label, xy=(x, y), xytext=(0, 0), textcoords='offset points')
    plt.xlim(x_coords.min()+0.00005, x_coords.max()+0.00005)
    plt.ylim(y_coords.min()+0.00005, y_coords.max()+0.00005)
    plt.show()

In [7]:
display_closestwords_tsnescatterplot(emb_model, 'sad')

MemoryError: Unable to allocate array with shape (3000000, 300) and data type float32

In [8]:
def phrase_pre_processing(phrase):
    # Basic Text PreProcessing
    phrase = re.sub(' +', ' ', re.sub(r'\d+', '', phrase.lower()).translate(str.maketrans("","", string.punctuation+'\n'+'\r'+'\t')))
    
    # Tokenizing
    tokens = word_tokenize(phrase)
    
    # Remove Stop Words
    stop_words = set(stopwords.words('english'))
    tokens = [i for i in tokens if not i in stop_words]
    
    # Lemmatization
    lemmatizer=WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(i) for i in tokens]

    return tokens

In [9]:
def phrase_to_embedding(tokens):
    words = [word for word in tokens if word in emb_model.vocab]
    embeddings = np.zeros(300)
    if len(words) >= 1:  
        avg = np.mean(emb_model[words], axis=0)
        for idx, e in enumerate(avg):
            embeddings.flat[idx] = e
            
    return embeddings

In [ ]:
X = []
y = []

for index, row in dataset.iterrows():
    tokens = phrase_pre_processing(row['Sentence'])
    embeddings = phrase_to_embedding(tokens)
    one_hot = np.zeros(7)
    one_hot.flat[emotion_labels[row['Sentiment']]] = 1
    y.append(one_hot)
    X.append(embeddings)

X = np.array(X)    
y = np.array(y)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1234)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

In [ ]:
train_X,valid_X,train_label,valid_label = train_test_split(X_train, y_train, test_size=0.2, random_state=1234)
train_X.shape,valid_X.shape,train_label.shape,valid_label.shape

In [ ]:
input_dim = X_train.shape[1]  # Number of features
batch_size = 64
epochs = 20

In [ ]:
model = Sequential()
model.add(layers.Dense(64, input_dim=input_dim, activation='relu'))
model.add(layers.Dense(128, activation='sigmoid'))
model.add(layers.Dense(7, activation='sigmoid'))

In [ ]:
model.compile(loss=binary_crossentropy, optimizer=Adam(), metrics=['accuracy'])

In [ ]:
model_train = model.fit(train_X, train_label, batch_size=batch_size, epochs=epochs, verbose=1, validation_data=(valid_X, valid_label))

In [ ]:
test_eval = model.evaluate(X_test, y_test, verbose=0)
print('Test loss:', test_eval[0])
print('Test accuracy:', test_eval[1])

In [ ]:
model.summary()

In [ ]:
accuracy = model_train.history['accuracy']
val_accuracy = model_train.history['val_accuracy']
loss = model_train.history['loss']
val_loss = model_train.history['val_loss']
epochs = range(len(accuracy))

In [ ]:
plt.plot(epochs, accuracy, 'bo', label='Training accuracy')
plt.plot(epochs, val_accuracy, 'b', label='Validation accuracy')
plt.xlabel("Epoch")
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.xlabel("Epoch")
plt.title('Training and validation loss')
plt.legend();

In [ ]:
test_sent = ""
tokens = phrase_pre_processing(test_sent)
embeddings = phrase_to_embedding(tokens)
idx = model.predict(embeddings.reshape(-1,300)).argmax()
for key, val in emotion_labels.items():
    if val == idx:
        print("Sentence Emotion Detector: ", key)